In [ ]:
pip install -U tensorflow_decision_forests

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt')
from sklearn.model_selection import train_test_split
import tensorflow_decision_forests as tfdf
import torch

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
df = pd.read_csv('/content/dialog_acts.dat')
df.columns = ['sentence']
df['description'] = df['sentence'].str.split(' ').str[0]
df['sentences'] = df['sentence'].str.split(n=1).str[1]
df['tokenized_sents'] = df.apply(lambda row: nltk.word_tokenize(row['sentences']), axis=1)

tokens = []
word_to_idx = {}
for each in df['tokenized_sents']:
  tokens.extend(each)
for i, word in enumerate(tokens):
    word_to_idx[word] = i

new_df = df[['description', 'tokenized_sents']].copy()
new_df

,description,tokenized_sents
0,inform,"[any, part, of, town]"
1,inform,"[bistro, food]"
2,confirm,"[is, there, a, moderately, priced, restaurant,..."
3,affirm,[yes]
4,request,"[could, i, get, their, phone, number]"
...,...,...
25495,request,"[what, is, the, price, range]"
25496,thankyou,"[thank, you, good, bye]"
25497,inform,"[im, looking, for, african, food]"
25498,request,"[whats, the, address, and, phone, number]"


In [ ]:
label = 'description'
classes = new_df[label].unique().tolist()
new_df[label] = new_df[label].map(classes.index)
numerical_values_list = []
numerical_tensors_list = []
tensor_cat_list = []

for each in new_df['tokenized_sents'].index:
    numerical_values = [word_to_idx[word] for word in new_df['tokenized_sents'][each]]
    numerical_values_list.append(numerical_values)
    numerical_tensors = torch.tensor(numerical_values)
    numerical_tensors_list.append(numerical_tensors)


new_df['numerical_tensors'] = numerical_tensors_list

new_df




 #   tokenized_tensors = torch.tensor(numerical_values)
 #   new_df.loc[each, 'tokenized_sents']= tokenized_tensors
#new_df
#df_to_train = new_df[['description', 'tokenized_tensors']]
#df_to_train

,description,tokenized_sents,numerical_tensors
0,0,"[any, part, of, town]","[tensor(94904), tensor(95057), tensor(95071), ..."
1,0,"[bistro, food]","[tensor(91026), tensor(95091)]"
2,1,"[is, there, a, moderately, priced, restaurant,...","[tensor(95079), tensor(95012), tensor(94992), ..."
3,2,[yes],[tensor(95026)]
4,3,"[could, i, get, their, phone, number]","[tensor(94331), tensor(94990), tensor(94983), ..."
...,...,...,...
25495,3,"[what, is, the, price, range]","[tensor(95078), tensor(95079), tensor(95093), ..."
25496,4,"[thank, you, good, bye]","[tensor(95098), tensor(95099), tensor(95100), ..."
25497,0,"[im, looking, for, african, food]","[tensor(95087), tensor(95088), tensor(95089), ..."
25498,3,"[whats, the, address, and, phone, number]","[tensor(95092), tensor(95093), tensor(95094), ..."


In [ ]:
tensor_list={}
for index, row in new_df.iterrows():
  tensor_value = new_df.iloc[index]['numerical_tensors']
  descr = new_df.iloc[index]['description']
  tensor_list[descr] = tensor_value
print(tensor_list)

{0: tensor([95087, 95088, 95089, 95090, 95091]), 1: tensor([95079, 94654, 94229, 95091]), 2: tensor([95026]), 3: tensor([95092, 95093, 95094, 95095, 95096, 95097]), 4: tensor([95098, 95099, 95100, 95101]), 5: tensor([95040]), 6: tensor([95100, 95101]), 7: tensor([94946, 94947, 94955, 95091]), 8: tensor([94600, 95068, 95093, 94843, 95057, 95071, 95072]), 9: tensor([94539]), 10: tensor([94526, 94975]), 11: tensor([92499, 92500, 94540]), 12: tensor([91192, 91193]), 13: tensor([79087]), 14: tensor([81649])}


using: https://www.tensorflow.org/decision_forests/tutorials/beginner_colab

In [ ]:
def split_dataset(df_to_train, test_ratio=0.30):
  """Splits a panda dataframe in two."""
  test_indices = np.random.rand(len(df_to_train)) < test_ratio
  return df_to_train[~test_indices], df_to_train[test_indices]

train_ds_pd, test_ds_pd = split_dataset(new_df)
print(train_ds_pd)

print("{} examples in training, {} examples for testing.".format(
    len(train_ds_pd), len(test_ds_pd)))

       description                            tokenized_sents  \
0                0                      [any, part, of, town]   
1                0                             [bistro, food]   
3                2                                      [yes]   
4                3      [could, i, get, their, phone, number]   
5                4                    [thank, you, good, bye]   
...            ...                                        ...   
25494            3                             [price, range]   
25495            3              [what, is, the, price, range]   
25497            0          [im, looking, for, african, food]   
25498            3  [whats, the, address, and, phone, number]   
25499            4                    [thank, you, good, bye]   

                                       numerical_tensors  
0      [tensor(94904), tensor(95057), tensor(95071), ...  
1                         [tensor(91026), tensor(95091)]  
3                                        [

In [ ]:
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, label=label)
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_ds_pd, label=label)

ValueError: ignored